<a href="https://colab.research.google.com/github/gimb99/gbgTPDI/blob/develop/ejercicioIntegrador/ejercicioIntegrador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>